# Voronoi Binned Radial Profile Examples 

Isaac Cheng - December 2021

Based on the [`stellar_mass_densities+vorbin_radial_profiles`
notebook](../galaxies/stellar_mass_densities/stellar_mass_densities+vorbin_radial_profiles.ipynb).

I highly recommend the reader to take a look at the
[`radial_profile_example.ipynb`](../packages/radial_profile_example.ipynb) in the
[`coop_f2021/packages`](../packages/) directory first.

Note that there are only 36 galaxies in this analysis because several VERTICO galaxies are
not in the NGVS footprint and I got the NGC 4189 flag map too late and NGC 4606's flag map
is too intrusive.



In [1]:
%cd "/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities/"  # change directory to where this file is

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import dill
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
# 
# Load my own packages
# 
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import plot_utils as pu
import radial_profile_utils as rpu
from radial_profile import RadialProfile
# 
GALDIST = 16.5 * u.Mpc  # distance to Virgo cluster centre

/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities


Below are all the galaxies in our NGVS-VERTICO sample. The arrays with the `HIGH_I` prefix
are arrays pertaining to high-inclination galaxies (i.e., $i \geq 80^\circ$). I will
explain more below.


In [2]:
# ----------------------------------------------------------------------------------------
GALAXIES = np.array(
    [
        "IC3392",
        # "NGC4189",  # ignored flag map, but bad results
        "NGC4192",
        "NGC4216",  # has unflagged foreground star
        "NGC4222",
        "NGC4254",
        "NGC4294",
        "NGC4298",
        "NGC4299",
        "NGC4302",
        "NGC4321",
        "NGC4330",
        "NGC4351",
        "NGC4380",
        "NGC4383",
        "NGC4388",
        "NGC4396",
        "NGC4402",
        "NGC4405",
        "NGC4419",
        "NGC4424",
        "NGC4450",
        "NGC4501",
        "NGC4522",
        "NGC4532",
        "NGC4535",
        "NGC4548",
        "NGC4567",
        "NGC4568",
        "NGC4569",
        "NGC4579",
        "NGC4580",
        # "NGC4606",  # flag map too intrusive
        "NGC4607",
        "NGC4651",
        "NGC4654",
        "NGC4689",
        "NGC4694",
    ]
)
INCLINATIONS = np.array(
    [
        68,
        # 42,  # ignored flag map, but bad results
        83,
        90,  # has unflagged foreground star
        90,
        39,
        74,
        52,
        14,
        90,
        32,
        90,
        48,
        61,
        56,
        83,
        83,
        80,
        46,
        74,
        61,
        51,
        65,
        82,
        64,
        48,
        37,
        49,
        70,
        69,
        40,
        46,
        # 69,  # flag map too intrusive
        90,
        53,
        61,
        38,
        62,
    ]
)  # degrees
POSITION_ANGLES = np.array(
    [
        219,
        # 70,  # ignored flag map, but bad results
        333,
        20,  # has unflagged foreground star
        238,
        243,
        151,
        132,
        128,
        356,
        280,
        238,
        251,
        158,
        17,
        271,
        304,
        270,
        18,
        131,
        274,
        170,
        320,
        35,
        159,
        12,
        318,
        251,
        211,
        203,
        273,
        337,
        # 38,  # flag map too intrusive
        2,
        75,
        300,
        341,
        323,
    ]
)  # degrees
# ----------------------------------------------------------------------------------------
# N.B. "high inclination" means inclination >= 80 degrees
HIGH_I_GALAXIES = np.array(
    [
        "NGC4192",
        "NGC4216",
        "NGC4222",
        "NGC4302",
        "NGC4330",
        "NGC4388",
        "NGC4396",
        "NGC4402",
        "NGC4522",
        "NGC4607",
    ]
)
HIGH_I_INCLINATIONS = np.array([83, 90, 90, 90, 90, 83, 83, 80, 82, 90,])  # degrees
HIGH_I_POSITION_ANGLES = np.array(
    [333, 20, 238, 356, 238, 271, 304, 270, 35, 2]
)  # degrees
# ----------------------------------------------------------------------------------------
HIGH_I_HEIGHTS_ARCSEC = np.array(
    [80, 60, 25, 55, 38, 65, 43, 45, 40, 70,]
)  # arcsec, for radial profiles
# ----------------------------------------------------------------------------------------
VCC_GALAXIES = np.array(  # the VCC number of each galaxy
    [
        1126,  # IC3392
        # 89,  # NGC4189, ignored flag map, but bad results
        92,  # NGC4192
        167,  # NGC4216, has unflagged foreground star
        187,  # NGC4222
        307,  # NGC4254
        465,  # NGC4294
        483,  # NGC4298
        491,  # NGC4299
        497,  # NGC4302
        596,  # NGC4321
        630,  # NGC4330
        692,  # NGC4351
        792,  # NGC4380
        801,  # NGC4383
        836,  # NGC4388
        865,  # NGC4396
        873,  # NGC4402
        874,  # NGC4405
        958,  # NGC4419
        979,  # NGC4424
        1110,  # NGC4450
        1401,  # NGC4501
        1516,  # NGC4522
        1554,  # NGC4532
        1555,  # NGC4535
        1615,  # NGC4548
        1673,  # NGC4567
        1676,  # NGC4568
        1690,  # NGC4569
        1727,  # NGC4579
        1730,  # NGC4580
        # 1859,  # NGC4606, flag map too intrusive
        1868,  # NGC4607
        -100,  # NGC4651 (EVCC number is 1102, cannot use EVCC number)
        1987,  # NGC4654
        2058,  # NGC4689
        2066,  # NGC4694
    ]
)

## 1. Determine height of rectangles for high-i galaxies

Since the NGVS images are not masked (like the VERTICO data), we cannot simply extend the
rectangular annuli used in our radial profiles to the edges of the image. Instead, we have
to manually set a rectangle height (i.e., the dimension perpendicular to the galaxy's
major axis).

I just chose heights that roughly cover the extent of the CO data. That is what the
function below is for.


You do not have to use the function below! Just choose an appropriate height for your
rectangles.


In [ ]:
def find_height(galaxy, i, pa, min_width, height, i_threshold=W80, snr_target=50):
    """
    This function just overlays 1 rectangle on an image of the galaxy + its CO contours.
    Also plots an RGB image just as a reference (since looking at only 1 band of an image
    can be misleading sometimes).

    See the radial profile example notebook (under the coop2021/packages/ folder) to see
    how to use my radial profile code.
    """
    #
    # Load CO data for contours
    #
    if galaxy == "NGC4321":  # no 9 arcsec data since native is 10 arcsec
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/native/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_round_mom0_Kkms-1.fits"
    else:
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_round_mom0_Kkms-1.fits"
    Ico_data, co_header = fits.getdata(Ico_path, header=True)
    co_wcs = WCS(co_header)
    centre = co_wcs.pixel_to_world(Ico_data.shape[1] / 2, Ico_data.shape[0] / 2)
    include_bad = True if i < i_threshold else False  # only include NaNs if low-i galaxy
    #
    # Load NGVS data (just need 1 band)
    #
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{galaxy}/"
    with open(galpath + f"{galaxy}_vorbin_SNR{snr_target}_ugizBinned.pkl", "rb") as f:
        file = dill.load(f)
        px_per_bin = file["px_per_bin"]
        iband_data = file["iband_signal"] / px_per_bin
        vorbin_wcs = file["wcs"]
        vorbin_wcs.array_shape = file["wcs_array_shape"]
        file = None  # free memory
    galaxy_rp = RadialProfile(iband_data, centre, i, pa, noise=None)
    galaxy_rp = galaxy_rp.calc_radial_profile(
        i_threshold=i_threshold,
        n_annuli=1,  # only 1 rectangle
        wcs=vorbin_wcs,
        min_width=min_width,
        include_bad=include_bad,
        func="mean",
        is_radio=False,
        high_i_height=height,
        bootstrap_errs=False,
    )
    fig, ax = plt.subplots(subplot_kw={"projection": vorbin_wcs})
    img = ax.imshow(galaxy_rp.data, cmap="magma_r", norm=mpl.colors.LogNorm())
    cbar = fig.colorbar(img)
    ax.plot(
        *vorbin_wcs.world_to_pixel(centre), "co", markersize=2, zorder=20
    )  # mark centre
    pu.add_annuli_RadialProfile(
        ax, galaxy_rp, alpha_coeff=0.25, zorder=10, ls="-", edgecolor="k", fc="k", lw=1
    )  # add rectangle
    ax.contour(  # overlay CO contours
        Ico_data,
        transform=ax.get_transform(co_wcs),
        levels=range(2),
        colors="w",
        linewidths=1,
    )
    ax.set_aspect("equal")
    ax.grid(False)
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.set_xlim(0, galaxy_rp.data.shape[1])
    ax.set_ylim(0, galaxy_rp.data.shape[0])
    ax.set_title(galaxy)
    plt.show()
    #
    # Make RGB plot to compare (see coop2021/galaxies/make_rgb.ipynb if you're confused)
    #
    # "red" channel (i-band)
    iband_path = (f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_i_data.fits")
    # "green" channel (g-band)
    gband_path = (f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_g_data.fits")
    # "blue" channel (u-band)
    uband_path = (f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_u_data.fits")
    iband_data_uncut, iband_header_uncut = fits.getdata(iband_path, header=True)
    gband_data_uncut, gband_header_uncut = fits.getdata(gband_path, header=True)
    uband_data_uncut, uband_header_uncut = fits.getdata(uband_path, header=True)
    iband_data, iband_wcs = fu.cutout_to_target(iband_data_uncut, WCS(iband_header_uncut), Ico_data, co_wcs)
    gband_data, _ = fu.cutout_to_target(gband_data_uncut, WCS(gband_header_uncut), Ico_data, co_wcs)
    uband_data, _ = fu.cutout_to_target(uband_data_uncut, WCS(uband_header_uncut), Ico_data, co_wcs)
    rgb_data = pu.lognorm_median(iband_data, gband_data, uband_data, a=1000, norm_factor=1000)
    fig, ax = plt.subplots(subplot_kw={"projection": iband_wcs})
    img = ax.imshow(rgb_data)
    ax.plot(*iband_wcs.world_to_pixel(centre), "ro", markersize=2, zorder=20)  # mark centre
    pu.add_annuli_RadialProfile(  # add rectangle
        ax, galaxy_rp, alpha_coeff=0.25, zorder=10, ls="-", edgecolor="w", fc="w", lw=1
    )
    ax.contour(  # overlay CO contours
        Ico_data,
        transform=ax.get_transform(co_wcs),
        levels=range(2),
        colors="w",
        linewidths=1,
    )
    ax.set_aspect("equal")
    ax.grid(False)
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.set_xlim(0, iband_data.shape[1])
    ax.set_ylim(0, iband_data.shape[0])
    ax.set_title(galaxy)
    ax.tick_params(color="w")
    plt.show()


One by one, manually set the `height` parameter so that the rectangle covers the galaxy.

These heights will be the heights used in our high-resolution, Voronoi-binned data's
radial profiles; they are stored in the `HIGH_I_HEIGHTS_ARCSEC` array in cell 2.


In [ ]:
# find_height(HIGH_I_GALAXIES[0], HIGH_I_INCLINATIONS[0], HIGH_I_POSITION_ANGLES[0], min_width=300*u.arcsec, height=80*u.arcsec)
# find_height(HIGH_I_GALAXIES[1], HIGH_I_INCLINATIONS[1], HIGH_I_POSITION_ANGLES[1], min_width=300*u.arcsec, height=60*u.arcsec)
# find_height(HIGH_I_GALAXIES[2], HIGH_I_INCLINATIONS[2], HIGH_I_POSITION_ANGLES[2], min_width=300*u.arcsec, height=25*u.arcsec)
# find_height(HIGH_I_GALAXIES[3], HIGH_I_INCLINATIONS[3], HIGH_I_POSITION_ANGLES[3], min_width=300*u.arcsec, height=55*u.arcsec)
# find_height(HIGH_I_GALAXIES[4], HIGH_I_INCLINATIONS[4], HIGH_I_POSITION_ANGLES[4], min_width=300*u.arcsec, height=38*u.arcsec)
# find_height(HIGH_I_GALAXIES[5], HIGH_I_INCLINATIONS[5], HIGH_I_POSITION_ANGLES[5], min_width=300*u.arcsec, height=65*u.arcsec)
# find_height(HIGH_I_GALAXIES[6], HIGH_I_INCLINATIONS[6], HIGH_I_POSITION_ANGLES[6], min_width=300*u.arcsec, height=43*u.arcsec)
# find_height(HIGH_I_GALAXIES[7], HIGH_I_INCLINATIONS[7], HIGH_I_POSITION_ANGLES[7], min_width=300*u.arcsec, height=45*u.arcsec)
# find_height(HIGH_I_GALAXIES[8], HIGH_I_INCLINATIONS[8], HIGH_I_POSITION_ANGLES[8], min_width=300*u.arcsec, height=40*u.arcsec)
# find_height(HIGH_I_GALAXIES[9], HIGH_I_INCLINATIONS[9], HIGH_I_POSITION_ANGLES[9], min_width=300*u.arcsec, height=70*u.arcsec)

## 2. Make radial profiles (in pickle file)

1. We set the minimum width of each annulus to be the FWHM of the largest seeing disk
   amongst our u-, g-, i-, and z-band data.
2. We determine the number of annuli to use based on the extent of the CO data. You can
   just specify your own number of annuli to use instead, if you want. <span
   style="color:red">**VERY IMPORTANT:**</span> I _do not_ recommend using an SNR cutoff
   to determine the number of annuli to use because the images are not masked and thus
   there will likely be thousands of annuli in the radial profile! This will take a very
   long time if you are bootstrapping errors and it will take a lot of memory (since I
   haven't implemented the option to reduce memory usage in my radial profile code yet).
   If you really really want to add annuli until the edges of the image, at least choose a
   larger minimum annulus width!



Load data containing radio annuli to use as reference for number of annuli to fit. You may
choose not to do this and specify the number of annuli to use manually


In [3]:
co_rad_profs_path = "/arc/home/IsaacCheng/coop_f2021/galaxies/gas_fraction/gas_fraction_i_corr_NGVS-VERTICO_noNorm.pkl"
with open(co_rad_profs_path, "rb") as f:
    file = dill.load(f)
    galaxies = file["galaxies"]
    inclinations = file["inclinations"]
    position_angles = file["position_angles"]
    centers = file["centers"]
    radial_profiles = file["radial_profiles"]
    annulus_widths_arcsec = file["annulus_widths_arcsec"]
    file = None  # free memory

avg_function = radial_profiles[0].rp_options["func"]
radio_n_annulis = []
for rp in radial_profiles:
    radio_n_annulis.append(len(rp.annuli))
print(radio_n_annulis)
print(galaxies)
# print(centers)

[3, 19, 21, 9, 16, 3, 7, 6, 18, 15, 9, 5, 5, 5, 10, 8, 13, 3, 4, 4, 7, 7, 7, 4, 12, 11, 6, 5, 8, 11, 4, 7, 6, 8, 8, 5]
['IC3392' 'NGC4192' 'NGC4216' 'NGC4222' 'NGC4254' 'NGC4294' 'NGC4298'
 'NGC4299' 'NGC4302' 'NGC4321' 'NGC4330' 'NGC4351' 'NGC4380' 'NGC4383'
 'NGC4388' 'NGC4396' 'NGC4402' 'NGC4405' 'NGC4419' 'NGC4424' 'NGC4450'
 'NGC4501' 'NGC4522' 'NGC4532' 'NGC4535' 'NGC4548' 'NGC4567' 'NGC4568'
 'NGC4569' 'NGC4579' 'NGC4580' 'NGC4607' 'NGC4651' 'NGC4654' 'NGC4689'
 'NGC4694']


Function to generate radial profile based on the number of annuli used for VERTICO data.

You will need to change the paths used for loading/writing data!


In [4]:
# ! MAKE SURE YOU RUN CELL 2 (the one with HIGH_I_HEIGHTS)

I_THRESHOLD = 80  # if inclination >= I_THRESHOLD, then use rectangular slices of galaxy in profile

def get_vorbin_rp(
    galaxy,  # galaxy name. string (e.g., "NGC4380")
    i,  # inclination. float or int
    pa,  # position angle. float or int
    center,  # centre of the galaxy. astropy SkyCoord
    radio_annulus_width,  # width of the radio annuli (i.e., 9"). must be a float in units of arcsec
    num_radio_annuli,  # number of radio annuli. int
    rp_quantity="M_density",  # quantity to calculate. "M_density", "MLi", or "u-g"
    snr_mask=False,  # if True, mask any regions that fail the snr_mask contained in the data's pickle file
    n_annuli=None,  # if not None, specify the number of annuli to use (and overrides internal n_annuli calculation)
    min_width=None,  # if not None, specify the minimum annulus width (and overrides internal min_width calculation). must be a float in units of arcsec
    avg_function="mean",  # "mean" or "median"
    include_bad=False,  # if True, include masked regions in the data as zeros
    snr_target=50,  # specifies the vorbin pickle file to use
    bootstrap_errs=True,  # if True, estimate the error in the mean using bootstrapping
    n_bootstraps=100,  # number of bootstrap iterations to use when estimating the error in the mean
    bootstrap_seed=1234,  # seed for bootstrapping
    avg_uncertainty=False,  # if True, the radial profile calculates the average uncertainty instead of the average value. Note that this is different from the uncertainty in the mean!
):
    """
    Calculate the radial profile of a galaxy.

    Be careful. I'm not doing any input checks
    """
    # 
    # Load stellar mass density data
    # 
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{galaxy}/"  # ! CHANGE ME
    galaxy_infile = galpath + f"{galaxy}_vorbin_SNR{snr_target}_extinctionCorr_ugiz_Sigma-star_i_corr.pkl"  # ! CHANGE ME
    with open(galaxy_infile, "rb") as f:
        file = dill.load(f)
        if rp_quantity == "M_density":
            rp_quantity = file ["stellar_mass_density"]
            rp_quantity_err = file["stellar_mass_density_err"]
        elif rp_quantity == "MLi":
            rp_quantity = file ["MLi"]
            rp_quantity_err = file["MLi_err"]
        elif rp_quantity == "u-g":
            uband_rel_mag = file ["uband_rel_mag"]
            uband_rel_mag_err = file["uband_rel_mag_err"]
            gband_rel_mag = file ["gband_rel_mag"]
            gband_rel_mag_err = file["gband_rel_mag_err"]
            rp_quantity = uband_rel_mag - gband_rel_mag
            rp_quantity_err = np.sqrt(uband_rel_mag_err ** 2 + gband_rel_mag_err ** 2)
            # Free memory
            uband_rel_mag = uband_rel_mag_err = None
            gband_rel_mag = gband_rel_mag_err = None
        else:
            raise ValueError(f"rp_quantity must be one of 'M_density', 'MLi', or 'u-g'")
        gal_wcs = file["wcs"]
        gal_wcs.array_shape = file["wcs_array_shape"]
        if snr_mask:
            isgood_snr = file["isgood_snr"]
            rp_quantity[~isgood_snr] = np.nan
            rp_quantity_err[~isgood_snr] = np.nan
            isgood_snr = None  # free memory
        file = None  # free memory
    #
    # Get radial profile parameters
    #
    if i >= I_THRESHOLD:
        high_i_height = HIGH_I_HEIGHTS_ARCSEC[galaxy == HIGH_I_GALAXIES]
        if high_i_height.size != 1:
            raise ValueError(f"Failed on {galaxy} with high_i_height={high_i_height}")
        high_i_height = high_i_height[0] * u.arcsec
    else:
        high_i_height = None
    print(f"{galaxy} high_i_height:", high_i_height)
    # 
    # Get worst image quality to use as min_width in radial profile
    # 
    if min_width is None:
        # Load data
        uband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_u_data.fits"  # ! CHANGE ME
        gband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_g_data.fits"  # ! CHANGE ME
        iband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_i_data.fits"  # ! CHANGE ME
        zband_path = f"/arc/home/IsaacCheng/coop_f2021/ngvs_data/{galaxy}/{galaxy}_z_data.fits"  # ! CHANGE ME
        # 
        uband_header = fits.getheader(uband_path)
        gband_header = fits.getheader(gband_path)
        iband_header = fits.getheader(iband_path)
        zband_header = fits.getheader(zband_path)
        # 
        header_order = ["u", "g", "i", "z"]
        header_lst = [uband_header, gband_header, iband_header, zband_header]
        min_width, min_width_idx = fu.get_worst_img_qual(
            header_lst, header_key="IQMAX", header_unit=u.arcsec
        )
        print(f"{galaxy}'s worst image quality is", min_width, f"from {header_order[min_width_idx]}-band data")
        min_width = min_width.to(u.arcsec).value
    if n_annuli is None:
        n_annuli = int(np.ceil((radio_annulus_width / min_width) * num_radio_annuli))
        print(f"INFO: Using {n_annuli} n_annuli")
    else:
        print(f"INFO: Using given n_annuli ({n_annuli}) instead of automatically-calculated n_annuli")
    #
    # Make radial profile
    #
    if avg_uncertainty:
        galaxy_rp = RadialProfile(rp_quantity_err, center, i, pa, noise=None)
    else:
        galaxy_rp = RadialProfile(rp_quantity, center, i, pa, noise=rp_quantity_err)
    galaxy_rp = galaxy_rp.calc_radial_profile(
        i_threshold=I_THRESHOLD,
        n_annuli=n_annuli,
        min_width=min_width * u.arcsec,
        wcs=gal_wcs,
        include_bad=include_bad,
        method="exact",
        func=avg_function,
        is_radio=False,
        high_i_height=high_i_height,
        bootstrap_errs=bootstrap_errs,
        n_bootstraps=n_bootstraps,
        bootstrap_seed=bootstrap_seed,
    )
    return galaxy_rp, n_annuli, min_width  # min_width is scalar in arcsec

Run function to make radial profiles of stellar mass density (`M_density`), mass-to-light
ratio (`MLi`) and u-g colour (`u-g`) for all NGVS-VERTICO galaxies listed in cell 2.

Then save each galaxy's radial profile to its own pickle file.

N.B. The pickle files are huge since I am saving a copy of the array for each data & noise
area mask!!! Will add option to reduce memory usage in the future so multiprocessing will
be an option.


In [ ]:
#
# Use for loop to get radial profiles for all galaxies. Read note above if you want to use
# multiprocessing...
#
quantity = "M_density"  # "M_density", "MLi", or "u-g"
use_snr_mask = False  # if True, use SNR mask for that galaxy (do not recommend)
use_snr_mask_str = "yesSNRmask" if use_snr_mask else "noSNRmask"

for gal, i, pa, cen, radio_width, radio_n_annuli in zip(galaxies, inclinations, position_angles, centers, annulus_widths_arcsec, radio_n_annulis):
    #
    # Do radial profile
    #
    gal_rp, gal_n_annuli, gal_min_width_arcsec = get_vorbin_rp(
        gal, i, pa, cen, radio_width, radio_n_annuli, rp_quantity=quantity, snr_mask=use_snr_mask
    )
    #
    # Pickle results
    #
    gal_dir_str = f"/arc/home/IsaacCheng/coop_f2021/galaxies/vorbin_radial_profiles/{gal}/"  # ! CHANGE ME
    gal_rp_outfile = gal_dir_str + f"{gal}_{quantity}_rps_NGVS-VERTICO_{use_snr_mask_str}_i_corr.pkl"  # ! CHANGE ME
    with open(gal_rp_outfile, "wb") as f:
        dill.dump(
            {
                "radial_profile_quantity": quantity,
                "used_snr_mask": use_snr_mask,
                "galaxy": gal,
                "inclinations": i,
                "position_angle": pa,
                "center": cen,
                "radial_profile": gal_rp,
                "n_annuli": gal_n_annuli,
                "min_width_arcsec": gal_min_width_arcsec,
            },
            f
        )
    print("Pickled", gal_rp_outfile)
print("DONE!")

## 3. Plot radial profiles from pickle files

Note because it may confuse some people: the `avg_noise` attribute of the radial profiles
is the sum of the pixel uncertainties in quadrature, divided by the area of the annulus
(including any specified edge effects). So this quantity does _not_ show the mean/median
uncertainty.

To get the average uncertainty in each annulus (not the uncertainty in the mean/median),
just re-run the radial profile code except set the `avg_uncertainty` flag to `True`. This
will pass the uncertainty array as the "data" for the radial profile, which will be a true
average. The `avg_noise` is for estimating the SNR of an annulus.


In [ ]:
import re
# Set these values. The first 2 parameters will. amongst other things, determine the
# pickle file to load
QUANTITY_TO_PLOT = "M_density" # "M_density", "MLi", or "u-g"
USE_SNR_MASK = False  # if True, use SNR mask for that galaxy (do not recommend)
cmap = "magma"  # "magma" for M_density and MLi, "plasma" for u-g
# 
if QUANTITY_TO_PLOT == "M_density":
    quantity_str = r"$\rm \Sigma_\star$ [$\rm M_\odot\; pc^{-2}$]"
elif QUANTITY_TO_PLOT == "MLi":
    quantity_str = r"$M_\star / L_i$ [$\rm M_\odot\; L_{i, \odot}$]"
elif QUANTITY_TO_PLOT == "u-g":
    quantity_str = "u-g Colour"
else:
    raise ValueError("QUANTITY_TO_PLOT must be 'M_density', 'MLi', or 'u-g'")
use_snr_mask_str = "yesSNRmask" if USE_SNR_MASK else "noSNRmask"
# 
for gal in galaxies:
    # Only re-run high-i galaxies (to fix their radii values)
    if gal not in HIGH_I_GALAXIES:
        continue
    print("Plotting", gal)
    #
    # Load radial profile data
    #
    gal_dir_str = f"/arc/home/IsaacCheng/coop_f2021/galaxies/vorbin_radial_profiles/{gal}/"  # ! CHANGE ME
    gal_rp_infile = gal_dir_str + f"{gal}_{QUANTITY_TO_PLOT}_rps_NGVS-VERTICO_{use_snr_mask_str}_i_corr.pkl"  # ! CHANGE ME
    with open(gal_rp_infile, "rb") as f:
        file = dill.load(f)
        gal_rp = file["radial_profile"]
        file = None  # free memory
    vorbin_wcs = gal_rp.rp_options["wcs"]
    px_to_kpc = fu.calc_pc_per_px(vorbin_wcs, GALDIST)[0][0] / 1000  # assume square pixels
    # 
    # Plot
    #
    fig = plt.figure(figsize=mpl.figure.figaspect(1.5))
    ax1 = fig.add_subplot(2, 1, 1, projection=vorbin_wcs)
    #
    # Plot data
    #
    if QUANTITY_TO_PLOT == "M_density":
        img1 = ax1.imshow(gal_rp.data, cmap=cmap, norm=mpl.colors.LogNorm())
        extend = "neither"
    elif QUANTITY_TO_PLOT == "MLi":
        img1 = ax1.imshow(gal_rp.data, cmap=cmap, vmax=3)
        extend = "max"
    else:
        img1 = ax1.imshow(gal_rp.data, cmap=cmap, vmax=None)
        extend = "neither"
    cbar1 = fig.colorbar(img1, fraction=0.045, extend=extend)
    cbar1.set_label(quantity_str)
    # Mark centre
    ax1.plot(*vorbin_wcs.world_to_pixel(gal_rp.center), "co", markersize=1)
    # Add annuli
    pu.add_annuli_RadialProfile(
        ax1, gal_rp, ls="-", edgecolor="c", fc="None", lw=0.25, zorder=2, alpha_coeff=-1
    )
    # Add scalebar
    pu.add_scalebar(ax1, vorbin_wcs, dist=GALDIST, color="w")
    ax1.tick_params(color="w")
    ax1.set_xlim(0, gal_rp.data.shape[1])
    ax1.set_ylim(0, gal_rp.data.shape[0])
    ax1.set_xlabel("RA (J2000)")
    ax1.set_ylabel("Dec (J2000)")
    ax1.grid(False)
    ax1.set_aspect("equal")
    #
    ax2 = fig.add_subplot(2, 1, 2)
    ebar2 = ax2.errorbar(
        x=gal_rp.radii * px_to_kpc,
        y=gal_rp.avg_data,
        yerr=gal_rp.avg_data_err,  # uncertainty in the mean/median from bootstrapping
        fmt="-o",
        markersize=2,
        color="k",
        ecolor="r",
        elinewidth=1,
        capsize=2,
    )
    ebar2[-1][0].set_linestyle("--")
    # 
    # Add name of galaxy
    # 
    high_i_str = "*" if gal_rp.i >= I_THRESHOLD else ""
    ax2.text(
        0.9,
        0.9,
        re.sub(r"(\d+)", " \\1", gal) + high_i_str,
        c="k",
        ha="right",
        transform=ax2.transAxes,
    )
    ax2.set_xlabel("Radius [kpc]")
    ax2.set_ylabel(gal_rp.rp_options["func"].capitalize() + " " + quantity_str)
    if QUANTITY_TO_PLOT == "M_density":
        ax2.semilogy()
    ax2.set_xlim(left=0)
    #
    fig.tight_layout(pad=3.5)
    fig.savefig(gal_dir_str + f"{gal}_vorbin_SNR50_{QUANTITY_TO_PLOT}_radProf_fromLookupTable_{use_snr_mask_str}_i_corr.pdf")  # ! CHANGE ME
    plt.close()
print("Done!")